In [1]:
from nltk.stem.porter import *
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import wordnet
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from datetime import datetime
import re
from collections import defaultdict
from __future__ import print_function
from sklearn.cross_validation import train_test_split
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from datetime import datetime
from sklearn.cross_validation import StratifiedShuffleSplit
np.random.seed(1337) 

class DFtransformer(object):
    def __init__(self,min_df=5):
        self.min_df = min_df
        self.Vocab = defaultdict(int)
        self.Vocab_ = {}
    def fit(self,sentence_lst):
        for sentence in sentence_lst:
            for word in sentence.split():
                self.Vocab[word] += 1
        print ("origenal vocab:",len(self.Vocab))
        self.Vocab = {k:v for k,v in self.Vocab.items() if self.Vocab[k]>=self.min_df}   
        i = 1
        for k in self.Vocab:
            self.Vocab_[k] = i
            i += 1
        print ("transformed vocab:",len(self.Vocab))
        
    def tranform(self,sentence_lst):
        ret_s_lst = []
        for s in sentence_lst:
            ret_s = np.array([self.Vocab_[w] for w in s.split() if w in self.Vocab_])
            #print ret_s
            ret_s_lst.append(ret_s)
        return np.array(ret_s_lst)

toker = TreebankWordTokenizer()
lemmer = wordnet.WordNetLemmatizer()
stemmer = PorterStemmer()

stop_w = set(stopwords.words('english'))
# w2v_model =  '../../../../media/lhc/B2DA42D1DA429191/word2vec_data/GoogleNews-vectors-negative300.bin'
# embedder = Word2Vec.load_word2vec_format(w2v_model,binary=True)

def str_lem(s):
    s = s.lower()
    s = (" ").join([z for z in re.findall('\w{2,}',s) if z not in stop_w])
    s = (" ").join([lemmer.lemmatize(z) for z in s.split(" ")])
    return s
def str_stem(s):
    s = s.lower()
    s = (" ").join([z for z in re.findall('\w{2,}',s) if z not in stop_w])
    s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
    return s
def apply_text_preprocessor(columns,str_process):
    rets = []
    for i,row in enumerate(columns,start=1):
#         if i%10000==0:
#             print i,
        rets.append(str_process(row))
    return rets

Using Theano backend.


In [2]:
fin = './data/news_o2.csv'
df_all = pd.read_csv(fin,encoding='utf8')
#df_all = df_all[df_all.target.notnull()]
df_all['title_stem'] = apply_text_preprocessor(df_all['title'],str_stem)
df_all['title_lem'] = apply_text_preprocessor(df_all['title'],str_lem)
df_all.head()

,Date,d1,d2,d3,date,title,ticker,title_stem,title_lem
0,2015-07-02,-0.006306,0.00272,-0.050633,2015-07-02,Alcoa (AA) to Shutter Pocos de Caldas Smelter ...,AA,alcoa aa shutter poco de calda smelter brazil ...,alcoa aa shutter pocos de caldas smelter brazi...
1,2015-07-02,-0.006306,0.00272,-0.050633,2015-07-02,Q2 Earnings Season in the Spotlight with Alcoa...,AA,q2 earn season spotlight alcoa report earn pre...,q2 earnings season spotlight alcoa report earn...
2,2015-07-02,-0.006306,0.00272,-0.050633,2015-07-02,Earnings will be a nail biter: Jack Ablin,AA,earn nail biter jack ablin,earnings nail biter jack ablin
3,2015-07-02,-0.006306,0.00272,-0.050633,2015-07-02,3 Beaten-Down Stocks That May Pay Off for Cont...,AA,beaten stock may pay contrarian investor,beaten stock may pay contrarian investor
4,NaN,NaN,NaN,NaN,2015-07-03,UPDATE 1-Japan Q3 aluminium premiums mostly se...,AA,updat japan q3 aluminium premium mostli set ye...,update japan q3 aluminium premium mostly set y...


In [3]:
target = 'd1'

df = df_all[df_all[target].notnull()]
dftransformer =  DFtransformer(min_df=5)
dftransformer.fit(df['title_lem'])
seq_lst = dftransformer.tranform(df['title_lem'])
maxlen = 20
X = sequence.pad_sequences(seq_lst,maxlen=maxlen)
y = (df[target].values>0).astype(np.int64)

origenal vocab: 24651
transformed vocab: 9290


In [8]:
max_features = 9290 + 1
batch_size = 32

for tr,va in StratifiedShuffleSplit(y,train_size=0.8,test_size=0.2,random_state=1024,n_iter=1):
    X_train = X[tr]
    y_train = y[tr]
    X_test = X[va]
    y_test = y[va]

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('X_train[0]:',X_train[0])
print('y_train[0]:',y_train[0])

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen, dropout=0.2))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

# for n in range(15):
#     model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1,
#               validation_data=(X_test, y_test),verbose=1)
#     score, acc = model.evaluate(X_train, y_train,verbose=1,
#                                 batch_size=batch_size)
#     print()
#     print('iter:',n+1)
#     print('Train score:', score)
#     print('Train accuracy:', acc)
#     score, acc = model.evaluate(X_test, y_test,verbose=1,
#                                 batch_size=batch_size)
#     print('Test score:', score)
#     print('Test accuracy:', acc)
    
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test, y_test),verbose=1)
score, acc = model.evaluate(X_train, y_train,verbose=1,
                            batch_size=batch_size)

X_train shape: (86389L, 20L)
X_test shape: (21598L, 20L)
X_train[0]: [   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
  425 2571 5021 1323 3524]
y_train[0]: 0
Build model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_4 (Embedding)          (None, 20, 128)       1189248     embedding_input_4[0][0]          
____________________________________________________________________________________________________
lstm_4 (LSTM)                    (None, 128)           131584      embedding_4[0][0]                
____________________________________________________________________________________________________
dense_3 (Dense)                  (None, 1)             129         lstm_4[0][0]                     
____________________________________________________________________________________________

In [9]:
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test, y_test),verbose=1)

Train on 86389 samples, validate on 21598 samples
Epoch 1/15
86389/86389 [==============================] - 154s - loss: 0.4372 - acc: 0.7748 - val_loss: 0.9278 - val_acc: 0.5826
Epoch 2/15
86389/86389 [==============================] - 150s - loss: 0.4276 - acc: 0.7816 - val_loss: 0.9182 - val_acc: 0.5847
Epoch 3/15
86389/86389 [==============================] - 154s - loss: 0.4229 - acc: 0.7839 - val_loss: 0.9446 - val_acc: 0.5896
Epoch 4/15
86389/86389 [==============================] - 160s - loss: 0.4162 - acc: 0.7899 - val_loss: 0.9845 - val_acc: 0.5846
Epoch 5/15
86389/86389 [==============================] - 162s - loss: 0.4119 - acc: 0.7913 - val_loss: 0.9842 - val_acc: 0.5860
Epoch 6/15
86389/86389 [==============================] - 160s - loss: 0.4062 - acc: 0.7943 - val_loss: 1.0254 - val_acc: 0.5824
Epoch 7/15
86389/86389 [==============================] - 162s - loss: 0.4011 - acc: 0.7975 - val_loss: 1.0340 - val_acc: 0.5853
Epoch 8/15
86389/86389 [=======================

In [10]:
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test, y_test),verbose=1)

Train on 86389 samples, validate on 21598 samples
Epoch 1/15
86389/86389 [==============================] - 178s - loss: 0.3757 - acc: 0.8133 - val_loss: 1.0933 - val_acc: 0.5904
Epoch 2/15
86389/86389 [==============================] - 163s - loss: 0.3696 - acc: 0.8183 - val_loss: 1.1055 - val_acc: 0.5876
Epoch 3/15
86389/86389 [==============================] - 178s - loss: 0.3668 - acc: 0.8192 - val_loss: 1.1141 - val_acc: 0.5899
Epoch 4/15
86389/86389 [==============================] - 169s - loss: 0.3690 - acc: 0.8197 - val_loss: 1.0855 - val_acc: 0.5870
Epoch 5/15
86389/86389 [==============================] - 169s - loss: 0.3665 - acc: 0.8208 - val_loss: 1.1175 - val_acc: 0.5911
Epoch 6/15
86389/86389 [==============================] - 157s - loss: 0.3640 - acc: 0.8212 - val_loss: 1.1313 - val_acc: 0.5901
Epoch 7/15
86389/86389 [==============================] - 149s - loss: 0.3609 - acc: 0.8227 - val_loss: 1.1231 - val_acc: 0.5902
Epoch 8/15
86389/86389 [=======================